# Importing Essential Libraries

In [1]:
import os, sys
import pandas as pd
import geopandas as gpd
import shapely
import datetime
import numpy as np
from tqdm import tqdm
from datetime import datetime

In [2]:
import bokeh as bkh
import bokeh.models as bkhm
import bokeh.palettes as bokeh_palettes
import bokeh.colors as bokeh_colors

In [3]:
from st_visualizer import st_visualizer
import express as viz_express
import geom_helper as viz_helper
import callbacks

In [4]:
pd.set_option('display.expand_frame_repr', False)

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Defining Global Variables

In [5]:
NOTEBOOK_URL='http://<NOTEBOOK_IP_ADDRESS>:<NOTEBOOK_PORT>'

  * ### Loading GeoLife Dataset

In [ ]:
gdf = pd.read_csv('./data/csv/geolife_trips_cleaned_v2_china_subset.csv', nrows=50000)
gdf = viz_helper.getGeoDataFrame_v2(gdf, crs='epsg:4326')

  * ### Creating Choropleth (Grid) Geometry

In [ ]:
bbox = np.array(gdf.total_bounds)

p1 = shapely.geometry.Point(bbox[0], bbox[3])
p2 = shapely.geometry.Point(bbox[2], bbox[3])
p3 = shapely.geometry.Point(bbox[2], bbox[1])
p4 = shapely.geometry.Point(bbox[0], bbox[1])

np1 = (p1.coords.xy[0][0], p1.coords.xy[1][0])
np2 = (p2.coords.xy[0][0], p2.coords.xy[1][0])
np3 = (p3.coords.xy[0][0], p3.coords.xy[1][0])
np4 = (p4.coords.xy[0][0], p4.coords.xy[1][0])

spatial_coverage = gpd.GeoDataFrame(gpd.GeoSeries(shapely.geometry.Polygon([np1, np2, np3, np4])), columns=['geom'], geometry='geom',  crs='epsg:4326')
spatial_coverage_cut = viz_helper.quadrat_cut_geometry(spatial_coverage, 0.7)
spatial_coverage_cut = gpd.GeoDataFrame(np.array(list(spatial_coverage_cut)).reshape(-1,1), geometry=0, crs='epsg:4326')

  * ### Classifying Area Proximity (i.e., Populate the Choropleth Map)

In [ ]:
cnt = viz_helper.classify_area_proximity(gdf.copy(), spatial_coverage_cut, compensate=True, verbose=True).area_id.value_counts()

spatial_coverage_cut.loc[cnt.index, 'count'] = cnt.values
spatial_coverage_cut.rename({0:'geom'}, axis=1, inplace=True)
spatial_coverage_cut.set_geometry('geom', inplace=True)

# 1. Simple Choropleth Map

In [ ]:
st_viz = st_visualizer(limit=len(spatial_coverage_cut))
st_viz.set_data(spatial_coverage_cut.dropna())

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', plot_height=540, tools="pan,box_zoom,lasso_select,wheel_zoom,previewsave,reset")
st_viz.add_map_tile('CARTODBPOSITRON')

st_viz.add_numerical_colormap('Viridis256', 'count', colorbar=True, cb_orientation='vertical', cb_location='right', label_standoff=12, border_line_color=None, location=(0,0))
st_viz.add_polygon(fill_color=st_viz.cmap, line_color=st_viz.cmap, fill_alpha=0.6, muted_alpha=0, legend_label=f'GPS Locations (Choropleth Map)')

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bkhm.WheelZoomTool)

st_viz.show_figures(notebook=True, notebook_url=NOTEBOOK_URL)

# 2. (Advanced) Integrating Filters to Choropleth Map

### Create a VISIONS instance for the Choropleth plot (main canvas)

In [ ]:
st_viz = st_visualizer(limit=len(spatial_coverage_cut))
st_viz.set_data(spatial_coverage_cut)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', plot_height=540, tools="pan,box_zoom,lasso_select,wheel_zoom,previewsave,reset")
st_viz.add_map_tile('CARTODBPOSITRON')

st_viz.add_numerical_colormap('Viridis256', 'count', colorbar=True, cb_orientation='vertical', cb_location='right', label_standoff=12, border_line_color=None, location=(0,0), nan_color=bokeh_colors.RGB(1,1,1,0))
st_viz.add_polygon(fill_color=st_viz.cmap, line_color=st_viz.cmap, fill_alpha=0.6, muted_alpha=0, legend_label=f'GPS Locations (Choropleth)')

### Create a VISIONS instance for keeping (& filtering) the original data

In [ ]:
data_points = st_visualizer(limit=len(gdf))
data_points.set_data(gdf)
data_points.set_figure(st_viz.figure)


categorical_name='label'

class Callback(callbacks.BokehFilters):
    def __init__(self, vsn_instance, widget):
        super().__init__(vsn_instance, widget)
        
        
    def callback_prepare_data(self, new_pts, ready_for_output):
        self.vsn_instance.canvas_data = new_pts

        if ready_for_output:
            cnt = viz_helper.classify_area_proximity(self.vsn_instance.canvas_data, st_viz.data, compensate=True, verbose=True).area_id.value_counts()            
            st_viz.canvas_data = st_viz.data.loc[cnt.index].copy()
            st_viz.canvas_data.loc[:, 'count'] = cnt.values               
           
            st_viz.canvas_data = st_viz.prepare_data(st_viz.canvas_data)

            low, high = st_viz.canvas_data[st_viz.cmap['field']].agg([np.min, np.max])
            st_viz.cmap['transform'].low = 0 if low == high else low
            st_viz.cmap['transform'].high = high
                    
            st_viz.source.data = st_viz.canvas_data.drop(st_viz.canvas_data.geometry.name, axis=1).to_dict(orient="list")

            # print ('Releasing Lock...')
            st_viz.canvas_data = None
            self.vsn_instance.canvas_data = None
            self.vsn_instance.aquire_canvas_data = None
        
        
    def callback(self, attr, old, new):
        self.callback_filter_data()

        cat_value = self.widget.value
        new_pts = self.get_data()

        # print (cat_value, categorical_name)
        if cat_value:
            new_pts = new_pts.loc[new_pts[categorical_name] == cat_value].copy()

        self.callback_prepare_data(new_pts, self.widget.id==self.vsn_instance.aquire_canvas_data)

        
data_points.add_categorical_filter(title='Vehicle', categorical_name=categorical_name, height_policy='min', callback_class=Callback)

### Camera, Lights, Action

In [ ]:
data_points.figure.legend.location = "top_left"
data_points.figure.legend.click_policy = "mute"
data_points.figure.toolbar.active_scroll = data_points.figure.select_one(bkhm.WheelZoomTool)

In [ ]:
data_points.show_figures(notebook=True, notebook_url=NOTEBOOK_URL)

# Plot Multiple Datasets at the Same Canvas

In [ ]:
df = pd.read_csv('./data/csv/ais_brest_jan_24.csv')
df.loc[:, 'datetime'] = pd.to_datetime(df.timestamp, unit='ms')

df_algn_inter = pd.read_csv('./data/csv/kafka_aligned_data_inter_dataset_ais_brest_jan_24.csv')
df_algn_inter.loc[:, 'datetime'] = pd.to_datetime(df_algn_inter.timestamp, unit='s')

In [ ]:
st_viz = st_visualizer(limit=len(df))
st_viz.set_data(df, sp_columns=['lon', 'lat'])

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', plot_height=540, plot_width=540, tools="pan,box_zoom,lasso_select,wheel_zoom,previewsave,reset")
st_viz.add_map_tile('CARTODBPOSITRON')

st_viz.add_glyph(glyph_type='circle', size=10, color='royalblue', alpha=0.5, fill_alpha=0.5, muted_alpha=0, legend_label=f'GPS Locations')
st_viz.add_hover_tooltips(tooltips = [('MMSI','@mmsi'), ('Datetime','@datetime{%Y-%m-%d %H:%M:%S.%3N}'), ('Coordinates','(@lon, @lat)')], formatters={'@datetime': 'datetime'}, renderers=st_viz.renderers)

In [ ]:
st_viz2 = st_visualizer(limit=len(df_algn_inter))
st_viz2.set_data(df_algn_inter, sp_columns=['lon', 'lat'])

st_viz2.set_figure(st_viz.figure)
st_viz2.create_source()

st_viz2.add_glyph(glyph_type='square', size=10, color='orangered', alpha=0.5, fill_alpha=0.5, muted_alpha=0, legend_label=f'GPS Locations (interpolated - Kafka)')
st_viz2.add_hover_tooltips(tooltips = [('MMSI','@mmsi'), ('Datetime','@datetime{%Y-%m-%d %H:%M:%S.%3N}'), ('Coordinates','(@lon, @lat)')], formatters={'@datetime': 'datetime'}, renderers=st_viz2.renderers)

In [ ]:
df_traj = viz_helper.create_linestring_from_points(viz_helper.getGeoDataFrame_v2(df.copy()), ['mmsi'])

st_viz3 = st_visualizer(limit=len(df_traj))
st_viz3.set_data(df_traj, sp_columns=['lon', 'lat'])

st_viz3.set_figure(st_viz2.figure)
st_viz3.create_source()

st_viz3.add_line(line_type='multi_line', line_width=5, line_color='limegreen', alpha=0.5, muted_alpha=0, legend_label=f'Vessel Trajectories')

In [ ]:
st_viz3.figure.legend.location = "top_left"
st_viz3.figure.legend.click_policy = "mute"
st_viz3.figure.toolbar.active_scroll = st_viz3.figure.select_one(bkhm.WheelZoomTool)
st_viz3.show_figures(notebook=True, notebook_url=NOTEBOOK_URL)

# Plot Multiple Datasets at Grid

In [58]:
df = pd.read_csv('./data/csv/ais_brest_jan_24.csv')
df.loc[:, 'datetime'] = pd.to_datetime(df.ts, unit='s')

df_algn_inter = pd.read_csv('./data/csv/kafka_aligned_data_inter_dataset_ais_brest_jan_24.csv')
df_algn_inter.loc[:, 'datetime'] = pd.to_datetime(df_algn_inter.ts, unit='s')

In [72]:
st_viz = st_visualizer(limit=1000)
st_viz.set_data(df)

tooltips = [('MMSI','@mmsi'), ('Datetime','@datetime'), ('Coordinates','(@lon, @lat)')]
viz_express.plot_points_on_map(st_viz, tools=['lasso_select'], tooltips=tooltips)

In [73]:
st_viz2 = st_visualizer(limit=500)
st_viz2.set_data(df_algn_inter)

tooltips = [('MMSI','@mmsi'), ('Datetime','@datetime'), ('Coordinates','(@lon, @lat)')]
viz_express.plot_points_on_map(st_viz2, tools=['lasso_select'], tooltips=tooltips, glyph_type='triangle', color='red')

In [74]:
st_viz2.figure.legend.location = "top_left"
st_viz2.figure.legend.click_policy = "mute"
st_viz2.figure.toolbar.active_scroll = st_viz2.figure.select_one(bkhm.WheelZoomTool)

In [ ]:
st_viz2.show_figures([[st_viz.figure, st_viz2.figure]], notebook=True, merge_tools=True, toolbar_location='right', notebook_url=NOTEBOOK_URL)

# Visualize Data with Map alongside DataFrame

In [ ]:
df = pd.read_csv('./data/csv/ais_brest_jan_24.csv')


st_viz = st_visualizer(limit=10000)
st_viz.set_data(df)


tooltips = [('Vessel ID','@mmsi'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='stretch_both', plot_height=540, tools="pan,box_zoom,lasso_select,wheel_zoom,previewsave,reset", tooltips=tooltips)
st_viz.add_map_tile('CARTODBPOSITRON')

st_viz.add_glyph(fill_alpha=0.6, muted_alpha=0, legend_label=f'GPS Locations (Choropleth Map)')


columns = [
        bkhm.TableColumn(field="mmsi", title="MMSI"),
        bkhm.TableColumn(field="status", title="Status"),
        bkhm.TableColumn(field="turn", title="Turn"),
        bkhm.TableColumn(field="speed", title="Speed"),
        bkhm.TableColumn(field="heading", title="Heading"),
        bkhm.TableColumn(field="lon", title="Longitude"),
        bkhm.TableColumn(field="lat", title="Latitude"),
        bkhm.TableColumn(field="ts", title="Timestamp"),
        bkhm.TableColumn(field="velocity", title="Velocity"),
        bkhm.TableColumn(field="course_over_ground", title="COG")
        ]
data_table = bkhm.DataTable(source=st_viz.source, columns=columns, width=400, height=280)


st_viz.show_figures([[st_viz.figure, data_table]], plot_width=1900, sizing_mode='stretch_both', notebook=True, notebook_url=NOTEBOOK_URL)